# 使用ResNet-50做图像分类

## 数据预处理

In [9]:
fileDataName='balloon'

filePath='./data/'+fileDataName
filePathNew=filePath+'/ok'
fileEnd='.png'


In [10]:
import os
from PIL import Image

files_bad=[]

#thumbnailSize=1200

files=os.listdir(filePath)

isExist=os.path.exists(filePathNew)
if isExist==False:
    os.makedirs(filePathNew)

for file in files: 
    if file.endswith(fileEnd): 
        _fileFullPath=os.path.join(filePath,file)
        newName=os.path.splitext(file)[0]
        
        #print newName
        newpath=os.path.join(filePathNew,newName+'.jpg')          
        try: 
            img = Image.open(_fileFullPath) 
            rgbImg=img.convert("RGB")
            #print rgbImg.mode  
            #w, h = img.size
            #img.thumbnail((thumbnailSize, thumbnailSize))
            #print img
            rgbImg.save(newpath)
                
        except (IOError, SyntaxError) as e:
            print('Bad file:', file)
            files_bad.append(newpath)

print len(files_bad)

0


## 引用 Turi Create 库

In [11]:
import turicreate as tc

## 加载数据

In [25]:
import turicreate as tc
import os

def loadData(type):
    
    path='./sframe/'+type+'.sframe'
    filePath='./data/'+type+'/ok'
    
    if os.path.exists(filePath+'/.DS_Store'):
        os.remove(filePath+'/.DS_Store')
        
    if os.path.exists(path):
        ref_data =  tc.SFrame(path)
        print 'has sframe'
        return ref_data
    else:
        # 读取图片
        ref_data  = tc.image_analysis.load_images(filePath)
        #给每一行编号
        ref_data = ref_data.add_row_number()
        # 保存数据,以方便下次使用
        ref_data.save('./sframe/'+type+'.sframe')
        print 'nohas sframe'
        return ref_data

## Label the dataset
In the following block of code we will labels the image in the dataset of **Egg** and **Soup** images. Then we will export it as an `SFrame` data object to use it for training the image classification model.

1. The first line of code loads the folder images content using the `image_analysis` property. 

2. The second line creates a _foodType_ key for each image in the dataset to specify whether it's an **Egg** or **Soup** based on which folder it's located in.

3. The third line exports the analyzed data as an `SFrame` object in order to use it while creating our image classifier.

4. The fourth line simply visualises the new labeled image into a large list.

**Note**:- You do not have to run the following block of code everytime you create a classifer, unless you changed/edited the dataset.

In [26]:
type1='cake'
type2='balloon'

data1=loadData(type1)
data1["type"]=type1
data2=loadData(type2)
data2["type"]=type2

data=data1.append(data2)
data.save("./sframe/"+type1+"_or_"+type2+".sframe")

#data["foodType"] = data["path"].apply(lambda path: "Cake" if "cake" in path else "balloon")
#data.save("egg_or_soup.sframe")
#data.explore()

nohas sframe
nohas sframe


## 读取标记好的数据集
In the following line of code we are loading the `SFrame` object that contains the images in our dataset with their labels.

In [27]:
type1='cake'
type2='balloon'

dataBuffer = tc.SFrame("./sframe/"+type1+"_or_"+type2+".sframe")

In [28]:
print dataBuffer

+----+-------------------------------+--------------------------+------+
| id |              path             |          image           | type |
+----+-------------------------------+--------------------------+------+
| 0  | /root/kerasStudy/05-turicr... |  Height: 312 Width: 500  | cake |
| 1  | /root/kerasStudy/05-turicr... |  Height: 428 Width: 690  | cake |
| 2  | /root/kerasStudy/05-turicr... | Height: 2783 Width: 3519 | cake |
| 3  | /root/kerasStudy/05-turicr... |  Height: 256 Width: 256  | cake |
| 4  | /root/kerasStudy/05-turicr... | Height: 2142 Width: 2534 | cake |
| 5  | /root/kerasStudy/05-turicr... |  Height: 292 Width: 425  | cake |
| 6  | /root/kerasStudy/05-turicr... | Height: 2000 Width: 2000 | cake |
| 7  | /root/kerasStudy/05-turicr... |  Height: 300 Width: 400  | cake |
| 8  | /root/kerasStudy/05-turicr... | Height: 852 Width: 1280  | cake |
| 9  | /root/kerasStudy/05-turicr... | Height: 667 Width: 1000  | cake |
+----+-------------------------------+-------------

## 创建训练集和测试集 我们按照2:8的比例随机切割数据

In [29]:
trainingBuffers, testingBuffers = dataBuffer.random_split(0.8)

## 训练图像分类模型
In the following line of code, we will create an image classifier and we'll feed it with the training data we have. 

In this example, the image classifer's architecture will be a state-of-the-art Residual Network with 50 layers, also known as **ResNet-50**.

Check out the official paper here: https://arxiv.org/abs/1512.03385.

In [30]:
model = tc.image_classifier.create(trainingBuffers, target="type", model="resnet-50")

Resizing images...
Performing feature extraction on resized images...
Completed 45/45


WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) '__image_features__' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| Iteration | Passes   | Step size | Elapsed Time | Training-accuracy |

+-----------+----------+-----------+--------------+-------------------+

| 1         | 3        | 0.022222  | 1.022583     | 1.000000          |

+-----------+----------+-----------+--------------+-------------------+

SUCCESS: Optimal solution found.

## Evaluate the test data to determine the model accuracy

In [31]:
evaluations = model.evaluate(testingBuffers)
print evaluations["accuracy"]

1.0


## Save the Turi Create model to retrieve it later

In [8]:
model.save("./my_model_file/"+type1+"_or_"+type2+"_model_file")

## Export the image classification model for Core ML

In [9]:
model.export_coreml(type1+"_or_"+type2+"_Classifier.mlmodel")

## 预测新的图片

In [45]:
#predictions = model.predict(testingBuffers)

_path='./data/test6.jpg'

_img_query  = tc.turicreate.Image(_path)
_img_SF=tc.SFrame({
        "id": ['test'],
        "path":[_path ],
        "image":[_img_query ]
        })

predictions = model.predict(_img_SF)   
print predictions

['balloon']
